Loading the datasets from zipped drive folder ! 

In [19]:
from google_drive_downloader.google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1T3R7FDi-2fgJCuxooteRt9K_W_p7CRoo', dest_path='/content/Dataset',unzip=True) #add file id which are the characters in the url of the drive link

#https://drive.google.com/file/d/128wwBM1xqg3_-b1jYorkqDLq6swMatk8/view?usp=sharing - Bengali to odia
#https://drive.google.com/file/d/1T3R7FDi-2fgJCuxooteRt9K_W_p7CRoo/view?usp=sharing - Male to Male voice

In [20]:
pip install pyworld

In [21]:
pip install pysptk

Importing all the libraries !

In [22]:
import os
import math
import glob
import librosa
import pyworld
import pysptk
import numpy as np
import matplotlib.pyplot as plot

### Paths of audio files

Input the path to the original and synthesized .wav files.

In [23]:
ORIGINAL_PATH= '/content/Male-to-Male-Voice/SM1'
SYNTHESIZED_PATH='/content/Male-to-Male-Voice/converted_SM2_Speech_in_SM1_Voice'

### Functions to load files

Using librosa to load wav files.

In [24]:
def load_wav(wav_file, sr):
    
    wav, _ = librosa.load(wav_file, sr=sr, mono=True)

    return wav

Saving the file paths in a list.

In [25]:
Org_paths = glob.glob(ORIGINAL_PATH+'/*')
Synth_paths = glob.glob(SYNTHESIZED_PATH+'/*')
Org_paths.sort()
Synth_paths.sort()
for i in range(len(Org_paths)):
  assert os.path.basename(Org_paths[i])==os.path.basename(Synth_paths[i])
  #print(os.path.basename(Org_paths[i]),"   ",os.path.basename(Synth_paths[i]))
#print(Org_paths)

Loading the original and synthesized wav files in a list using load_wav function.

In [26]:
sampling_rate = 16000
num_mcep = 24
frame_period = 5.0
n_frames = 128
wavs_org=[]
wavs_synth=[]

for i in range(len(Org_paths)):
  if os.path.basename(Org_paths[i])==os.path.basename(Synth_paths[i]):
    wavs_org.append(load_wav(wav_file = Org_paths[i], sr = sampling_rate))
    wavs_synth.append(load_wav(wav_file = Synth_paths[i], sr = sampling_rate))

print(len(wavs_org))

162


### WORLD Analyzer (function to calculate and load F0 feature)

Using pyworld's harvest method to get the F0 values for each wav file and storing them in a list.

In [27]:
def world_encode_data(wavs, fs, frame_period = 5.0, coded_dim = 24):

    f0s = []
    
    num_mcep = 24
    for i in range(len(wavs)):
        wav = wavs[i]
        wav = wav.astype(np.float64)
        f0, _  = pyworld.harvest(wav, fs, frame_period = frame_period, f0_floor = 71.0, f0_ceil = 800.0)
        f0s.append(f0)
        #log_f0s_concatenated0 = np.ma.log(np.concatenate(f0s))
        #print(len(f0))
        

    return f0s#log_f0s_concatenated0

Loading the F0 values of original and synthesized files.



In [28]:

f0s_org = world_encode_data(wavs = wavs_org, fs = sampling_rate, frame_period = frame_period, coded_dim = num_mcep)
f0s_synth = world_encode_data(wavs = wavs_synth, fs = sampling_rate, frame_period = frame_period, coded_dim = num_mcep)

### Using dynamic time warping (DTW) for F0 RMSE calculation

Librosa's DTW is used for dynamic time warping.

In [29]:
min_cost_tot=[]
for i in range(len(wavs_org)):
  frame_len=0
  def logf0_rmse(x, y): # method to calculate cost
    #y=pad_to(y,len(x))
    log_spec_dB_const = 1/len(frame_len)
    # print(y)
    diff = x - y
    # print(x,"  ",y,"  ",len(y))
    # print(diff)
    #print(log_spec_dB_const * math.sqrt(np.inner(diff, diff)))
    return log_spec_dB_const * math.sqrt(np.inner(diff, diff))
  
  
  if len(f0s_org[i])<len(f0s_synth[i]):
    frame_len=f0s_org[i]
  else:
    frame_len=f0s_synth[i]

  cost_function = logf0_rmse
  min_cost, _ = librosa.sequence.dtw(f0s_org[i][:].T, f0s_synth[i][:].T, 
                                                   metric=cost_function)
  #print(len(min_cost))
  
  min_cost_tot.append(np.mean(min_cost))
   

In [31]:
F0RMSE=sum(min_cost_tot)/len(min_cost_tot)
print(f"F0_RMSE = {F0RMSE}")

F0_RMSE = 15.699971688352361
